In [2]:
# import required dependencies.
import numpy as np
import pandas as pd
import torch
from typing import Optional, List, Dict
from sklearn.preprocessing import LabelEncoder
from torch_choice.data import ChoiceDataset, JointDataset

# 
This tutorial aim to show how to manage choice datasets in the `torch-choice` package, we will follow the Stata documentation [here](https://www.stata.com/manuals/cm.pdf) to offer a seamless experience for the user to transfer prior knowledge in other packages to our package.

*From Stata Documentation*: Choice models (CM) are models for data with outcomes that are choices. The choices are selected by a decision maker, such as a person or a business, from a set of possible alternatives. For instance, we could model choices made by consumers who select a breakfast cereal from several different brands. Or we could model choices made by businesses who chose whether to buy TV, radio, Internet, or newspaper advertising.

Models for choice data come in two varieties—models for discrete choices and models for rankordered alternatives. When each individual selects a single alternative, say, he or she purchases one box of cereal, the data are discrete choice data. When each individual ranks the choices, say, he or she orders cereals from most favorite to least favorite, the data are rank-ordered data. Stata has commands for ﬁtting both discrete choice models and rank-ordered models.

Our `torch-choice` package handles the **discrete choice** models in the Stata document above.

## Data Layout

Typically in Stata, a single Stata observation corresponds to a single statistical observation—that is why Stata calls rows in a Stata dataset “observations”.

So as not to confuse statistical observations with Stata observations, we call a single statistical observation a **“case”** or a **"purchase record"** and use this terminology throughout the CM manual. 

In [3]:
df = pd.read_stata('https://www.stata-press.com/data/r17/carchoice.dta')

In [4]:
df

,consumerid,car,purchase,gender,income,dealers
0,1,American,1,Male,46.699997,9
1,1,Japanese,0,Male,46.699997,11
2,1,European,0,Male,46.699997,5
3,1,Korean,0,Male,46.699997,1
4,2,American,1,Male,26.100000,10
...,...,...,...,...,...,...
3155,884,Japanese,1,Male,20.900000,10
3156,884,European,0,Male,20.900000,4
3157,885,American,1,Male,30.600000,10
3158,885,Japanese,0,Male,30.600000,5


In [5]:
df_main = df[['consumerid', 'car', 'purchase']]
user_gender = pd.get_dummies(df.groupby('consumerid')['gender'].first().to_frame())
user_income = df.groupby('consumerid')['income'].first().to_frame()
price_dealers = df.set_index(['consumerid', 'car'])['dealers'].to_frame()

In [102]:
df_main.head()

,consumerid,car,purchase
0,1,American,1
1,1,Japanese,0
2,1,European,0
3,1,Korean,0
4,2,American,1


In [103]:
user_gender.head()

,gender_Female,gender_Male
consumerid,,
1,0,1
2,0,1
3,0,1
4,1,0
5,0,1


In [104]:
user_income.head()

,income
consumerid,
1,46.699997
2,26.100000
3,32.700001
4,49.199997
5,24.299999


In [82]:
class EasyDatasetWrapper():
    SUPPORTED_FORMATS = ['stata']
    def __init__(self,
                 main_data: pd.DataFrame,
                 case_index_column: str,  # also known as the purchase record.
                 alternative_column: str,
                 choice_column: str,  # purchase, binary.
                 user_index_column: Optional[str] = None,  # consumer idx.
                 session_index_column: Optional[str] = None,
                 user_observable_data: Optional[Dict[str, pd.DataFrame]] = dict(),
                 item_observable_data: Optional[Dict[str, pd.DataFrame]] = dict(),
                 session_observable_data: Optional[Dict[str, pd.DataFrame]] = dict(),
                 price_observable_data: Optional[Dict[str, pd.DataFrame]] = dict(),
                 format: str = 'stata'):

        if format not in self.SUPPORTED_FORMATS:
            raise ValueError(f'Format {format} is not supported, only {self.SUPPORTED_FORMATS} are supported.') 
        
        self.raw_data = main_data.copy()
        self.main_data = main_data
    
        # check compatibility of the data.
        # TODO:
        self.case_index_column = case_index_column
        self.case_index = main_data[case_index_column].unique()
        self.alternative_column = alternative_column
        self.choice_column = choice_column
        self.user_index_column = user_index_column
        self.session_index_column = session_index_column
        
        self.encode()

        self.align_observable_data(item_observable_data, user_observable_data, session_observable_data, price_observable_data)
        
        self.observable_data_to_observable_tensors()
        
        self.create_choice_dataset_from_stata()

    def encode(self) -> None:
        """Encodes item/alternative names, user names, and session names to {0, 1, 2, ...} integers."""
        self.item_name_encoder = LabelEncoder().fit(self.main_data[self.alternative_column].unique())
        
        if self.user_index_column is not None:
            self.user_name_encoder = LabelEncoder().fit(self.main_data[self.user_index_column].unique())
        
        if self.session_index_column is not None:
            self.session_name_encoder = LabelEncoder().fit(self.main_data[self.session_index_column].unique())

    def align_observable_data(self, item_observable_data, user_observable_data, session_observable_data, price_observable_data) -> None:
        self.item_observable_data = dict()
        for key, val in item_observable_data.items():
            self.item_observable_data['item_' + key] = val.loc[self.item_name_encoder.classes_]
        
        self.user_observable_data = dict() 
        for key, val in user_observable_data.items():
            self.user_observable_data['user_' + key] = val.loc[self.user_name_encoder.classes_]
    
        self.session_observable_data = dict()
        for key, val in session_observable_data.items():
            self.session_observable_data['session_' + key] = val.loc[self.session_name_encoder.classes_]
        
        self.price_observable_data = dict()
        for key, val in price_observable_data.items():
            # we will reshape price observable data to the observable_data_to_observable_tensors stage.
            self.price_observable_data['price_' + key] = val

    def observable_data_to_observable_tensors(self):
        """Convert all self.*_observable_data to self.*_observable_tensors for PyTorch."""
        self.item_observable_tensors = dict()
        for key, val in self.item_observable_data.items():
            self.item_observable_tensors[key] = torch.tensor(val.loc[self.item_name_encoder.classes_].values, dtype=torch.float32)

        self.user_observable_tensors = dict()
        for key, val in self.user_observable_data.items():
            self.user_observable_tensors[key] = torch.tensor(val.loc[self.user_name_encoder.classes_].values, dtype=torch.float32)
        
        self.session_observable_tensors = dict()
        for key, val in self.session_observable_data.items():
            self.session_observable_tensors[key] = torch.tensor(val.loc[self.session_name_encoder.classes_].values, dtype=torch.float32)
        
        self.price_observable_tensors = dict()
        for key, val in self.price_observable_data.items():
            val = val.copy()

            column_list = val.columns
            complete_index = pd.MultiIndex.from_product([self.session_name_encoder.classes_, self.item_name_encoder.classes_],
                                                        names=[self.session_index_column, self.alternative_column])
            val = val.reindex(complete_index)
            # convert item index and session names to the encoded values, and add indices to columns.
            val = val.reset_index()
            val[self.session_index_column] = self.session_name_encoder.transform(val[self.session_index_column].values)
            val[self.alternative_column] = self.item_name_encoder.transform(val[self.alternative_column].values)

            tensor_slice = list()
            for column in column_list:
                df_slice = val.pivot(index=self.session_index_column, columns=self.alternative_column, values=column)
                
                assert np.all(df_slice.index == np.arange(len(self.session_name_encoder.classes_)))
                assert np.all(df_slice.columns == np.arange(len(self.item_name_encoder.classes_)))
                tensor_slice.append(torch.Tensor(df_slice.values).float())

            tensor = torch.stack(tensor_slice, dim=-1)
            self.price_observable_tensors[key] = tensor

    def create_choice_dataset_from_stata(self):
        print('Creating choice dataset from stata format data-frames...')
        choice_set_size = self.main_data.groupby(self.case_index_column)[self.alternative_column].nunique()
        s = choice_set_size.value_counts()
        rep = dict(zip([f'size {x}' for x in s.index], [f'occurrence {x}' for x in s.values]))
        if len(np.unique(choice_set_size)) > 1:
            print(f'Note: choice sets of different sizes found in different purchase records: {rep}')
            self.item_availability = self.get_item_availability_tensor()
 
        item_bought = self.main_data[self.main_data[self.choice_column] == 1].set_index(self.case_index_column).loc[self.case_index, self.alternative_column].values
        self.item_index = self.item_name_encoder.transform(item_bought)

        # user index
        if self.user_index_column is None:
            self.user_index = None
        else:
            # get the user index of each purchase record.
            self.user_index = self.main_data.groupby(self.case_index_column)[self.user_index_column].first().loc[self.case_index].values
            self.user_index = self.user_name_encoder.transform(self.user_index)

        # session index
        if self.session_index_column is None:
            # print('Note: no session index provided, assign each case/purchase record to a unique session index.')
            self.session_index = None
        else:
            self.session_index = self.session_name_encoder.transform(self.main_data.groupby(self.case_index_column)[self.session_index_column].first().loc[self.case_index].values)
        
        self.choice_dataset = ChoiceDataset(item_index=torch.LongTensor(self.item_index),
                                            user_index=torch.LongTensor(self.user_index) if self.user_index is not None else None,
                                            session_index=torch.LongTensor(self.session_index) if self.session_index is not None else None,
                                            item_availability=self.item_availability,
                                            **self.item_observable_tensors,
                                            **self.user_observable_tensors,
                                            **self.session_observable_tensors,
                                            **self.price_observable_tensors)

    def get_item_availability_tensor(self) -> torch.BoolTensor:
        if self.session_index_column is None:
            raise ValueError(f'Item availability cannot be constructed without session index column.')
        A = self.main_data.pivot(self.session_index_column, self.alternative_column, self.choice_column)
        return torch.BoolTensor(~np.isnan(A.values))

    def __len__(self):
        return len(self.item_index)
    
    def summary(self):
        print(f'* Space of {len(self.item_name_encoder.classes_)} items:\n', pd.DataFrame(data={'item name': self.item_name_encoder.classes_}, index=np.arange(len(self.item_name_encoder.classes_))).T)
        print(f'* Number of purchase records/cases: {len(self)}.')
        print('* Preview of main data frame:')
        print(self.main_data)
        print('* Preview of ChoiceDataset:')
        print(self.choice_dataset)

In [83]:
A = data.main_data.pivot('consumerid', 'car', 'purchase').values
(~np.isnan(A)).astype(int)

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       ...,
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 0]])

In [84]:
data = EasyDatasetWrapper(df_main,
                          case_index_column='consumerid',
                          alternative_column='car',
                          choice_column='purchase',
                          session_index_column='consumerid',
                          user_index_column='consumerid',
                          user_observable_data={'gender': user_gender, 'income': user_income},
                          price_observable_data={'dealer': price_dealers})

Creating choice dataset from stata format data-frames...
Note: choice sets of different sizes found in different purchase records: {'size 4': 'occurrence 505', 'size 3': 'occurrence 380'}


In [85]:
# Use summary to see what's inside the data wrapper.
data.summary()

* Space of 4 items:
                   0         1         2       3
item name  American  European  Japanese  Korean
* Number of purchase records/cases: 885.
* Preview of main data frame:
      consumerid       car  purchase
0              1  American         1
1              1  Japanese         0
2              1  European         0
3              1    Korean         0
4              2  American         1
...          ...       ...       ...
3155         884  Japanese         1
3156         884  European         0
3157         885  American         1
3158         885  Japanese         0
3159         885  European         0

[3160 rows x 3 columns]
* Preview of ChoiceDataset:
ChoiceDataset(label=[], item_index=[885], user_index=[885], session_index=[885], item_availability=[885, 4], user_gender=[885, 2], user_income=[885, 1], price_dealer=[885, 4, 2], device=cpu)


In [75]:
len(data)

885